In [14]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [15]:
dfgN = gpd.read_file('data/NeighborhoodTabulationAreas.geojson')
dfgB = gpd.read_file('data/BoroughBoundaries.geojson')
dfgN.columns = map(str.lower, dfgN.columns)
dfgN = dfgN.applymap(lambda s:s.lower() if type(s) == str else s)

dfgB.columns = map(str.lower, dfgB.columns)
dfgB = dfgB.applymap(lambda s:s.lower() if type(s) == str else s)

In [16]:
dfgN.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,bk88,54005019.048,047,borough park,39247.2278309,brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,qn51,52488277.846,081,murray hill,33266.9047967,queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,qn27,19726845.6282,081,east elmhurst,19816.7120736,queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,qn07,22887772.8652,081,hollis,20976.3355739,queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
4,mn06,10647077.5264,061,manhattanville,17040.6854129,manhattan,1,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ..."


In [17]:
dfgB.rename(columns={'boro_name': 'borough'}, inplace=True)
dfgB.head()

,boro_code,borough,shape_area,shape_leng,geometry
0,2,bronx,1186612478.27,462958.187578,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
1,5,staten island,1623757282.78,325956.009,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
2,3,brooklyn,1937593022.64,738745.842115,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."
3,4,queens,3045878293.21,904188.424488,"MULTIPOLYGON (((-73.83668 40.59495, -73.83678 ..."
4,1,manhattan,636602658.887,361212.476098,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."


In [18]:
full_dfN = pd.read_csv('data/Trees_Properties_With_Centroids.csv')
full_dfB = pd.read_csv('data/Trees_Properties_With_Centroids_Boro.csv')
full_dfNP =full_dfN
full_dfBP = full_dfB################stopped here

In [19]:
full_dfB.head()

,Unnamed: 0,borough,area,health,trees/sq.mile,avg.landprice_thous$/acre,properties/sq.mile,centerLong,centerLat,hover
0,0,bronx,34.29,13.886667,2582.143056,58.444444,2665.613056,-73.8648,40.8448,bronx<br>trees/sq.mile: 2582.14<br>avg. landpr...
1,1,brooklyn,69.47,13.874510,2844.993137,154.762157,5497.278235,-73.9442,40.6782,brooklyn<br>trees/sq.mile: 2844.99<br>avg. lan...
2,2,manhattan,18.79,13.418077,3210.450385,257.268077,7584.695000,-73.9712,40.7831,manhattan<br>trees/sq.mile: 3210.45<br>avg. la...
3,3,queens,116.28,13.855085,2545.812881,209.636949,4061.352373,-73.7949,40.7282,queens<br>trees/sq.mile: 2545.81<br>avg. landp...
4,4,staten island,57.39,13.924444,2130.842778,24.254444,2752.206111,-74.1502,40.5795,staten island<br>trees/sq.mile: 2130.84<br>avg...


Add bins column

<h3>Bins for neighborhoods</h3>

In [20]:
binsN_trees = [
    "0-500",
    "501-1000",
    "1001-1500",
    "1501-2000",
    "2001-2500",
    "2501-3000",
    "3001-3500",
    "3501-4000",
    "4001-4500",
    "4501-5000"
]
binsB_trees = [
    "2000-2300",
    "2301-2600",
    "2601-2900",
    "2901-3200",
    "3201-3500"
]

<h3>Bins for boroughs</h3>

In [21]:

binsN_prop = [
    "0-21",
    "21-40",
    "41-60",
    "61-80",
    "81-100",
    "above 100"
]
binsB_prop = [
    "0-50",
    "51-100",
    "101-150",
    "151-200",
    "above 200"
]

In [22]:
full_dfN['bin']=full_dfN['trees/sq.mile'].apply(lambda x: binsN_trees[0] if int(x) <=500 else binsN_trees[1] if int(x) <=1000 else binsN_trees[2] if x <=1500 else binsN_trees[3] if int(x) <=2000 else binsN_trees[4] if int(x) <=2500 else binsN_trees[5] if int(x) <=3000 else binsN_trees[6] if int(x) <=3500 else binsN_trees[7] if int(x) <=4000 else binsN_trees[8] if int(x) <=4500 else binsN_trees[9] )
full_dfB['bin']=full_dfB['trees/sq.mile'].apply(lambda x: binsB_trees[0] if int(x) <=2300 else binsB_trees[1] if int(x) <=2600 else binsB_trees[2] if x <=2900 else binsB_trees[3] if int(x) <=3200 else binsB_trees[4])


In [23]:
full_dfNP['bin_p']=full_dfN['avg.landprice_thous$/acre'].apply(lambda x: binsN_prop[0] if int(x) <=20 else binsN_prop[1] if int(x) <=40 else binsN_prop[2] if int(x) <=60 else binsN_prop[3] if int(x) <=80 else binsN_prop[4] if int(x) <=100 else binsN_prop[5])
full_dfBP['bin_p']=full_dfB['avg.landprice_thous$/acre'].apply(lambda x: binsB_prop[0] if int(x) <=50 else binsB_prop[1] if int(x) <=100 else binsB_prop[2] if int(x) <=150 else binsB_prop[3] if int(x) <=200 else binsB_prop[4] )

<h3> Create geojson files according to bins</h3>

In [24]:
merged = dfgN.merge(full_dfN, on='ntaname' )
for bin in binsN_trees:
    print(bin)
    geo_layer = merged[(merged['bin'] == bin)] # geo
    try:
        geo_layer.to_file("data/neighborhoods/trees_per_sqmile/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

0-500
501-1000
1001-1500
1501-2000
2001-2500
2501-3000
3001-3500
3501-4000
4001-4500
4501-5000


In [25]:
mergedB = dfgB.merge(full_dfB, on='borough' )
for bin in binsB_trees:
    print(bin)
    geo_layer = mergedB[(mergedB['bin'] == bin)] # geo
    try:
        geo_layer.to_file("data/boroughs/trees_per_sqmile/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

2000-2300
2301-2600
2601-2900
2901-3200
err
3201-3500


In [26]:
mergedP = dfgN.merge(full_dfNP, on='ntaname' )
for bin in binsN_prop:
    print(bin)
    geo_layer = mergedP[(mergedP['bin_p'] == bin)] # geo
    try:
        geo_layer.to_file("data/neighborhoods/land_price/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

0-21
21-40
41-60
61-80
81-100
above 100


In [27]:
mergedPB = dfgB.merge(full_dfBP, on='borough' )
for bin in binsB_prop:
    print(bin)
    geo_layer = mergedPB[(mergedPB['bin_p'] == bin)] # geo
    try:
        geo_layer.to_file("data/boroughs/land_price/{}.geojson".format(bin), driver='GeoJSON')
    except:
        print('err')

0-50
51-100
101-150
err
151-200
above 200
